In [14]:
forest_data <- read_tsv("data/Algerian_forest_fires_dataset_UPDATE.csv", col_names = FALSE, skip = 2)
forest_data

Parsed with column specification:
cols(
  X1 = col_character()
)



X1
<chr>
"01,06,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire"
"02,06,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire"
"03,06,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire"
"04,06,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire"
"05,06,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire"
"06,06,2012,31,67,14,0,82.6,5.8,22.2,3.1,7,2.5,fire"
"07,06,2012,33,54,13,0,88.2,9.9,30.5,6.4,10.9,7.2,fire"
"08,06,2012,30,73,15,0,86.6,12.1,38.3,5.6,13.5,7.1,fire"
"09,06,2012,25,88,13,0.2,52.9,7.9,38.8,0.4,10.5,0.3,not fire"
